In [1]:
import os
import pandas as pd
import numpy as np
# import molvs

# Data Cleaning

In [ ]:
# Data V1
df = pd.read_csv('Consensus_CompoundBioactivity_Dataset_v1.1.csv')
df = df[df['Unit'] == 'neg. log']
df = df[df['Activity type'].isin(['pIC50', 'pPotency', 'pKi', 'pKd', 'pEC50'])]
df.to_csv('filtered_activity.csv', index=None)

In [ ]:
# Data V2
def find_suit_value(data, idx):
    st = np.where(np.array(data.columns)=='Mean_C (0)')[0][0]
    ed = np.where(np.array(data.columns)=='Mean_PD (9)')[0][0]
    ac_value = data.iloc[idx, st:ed].dropna().values
    sort_value = []
    for value in ac_value:
        v = float(value.split(' ')[0])
        count = int(value.split('(')[1].split(')')[0])
        sort_value.append([count, v])
    sort_value = np.array(sorted(sort_value, key=lambda x: x[0], reverse=True))
    max_count_values = sort_value[np.where(sort_value[:, 0] == sort_value[:, 0].max()), 1]
    if max_count_values.max() - max_count_values.min() >= 1:
        return None
    return np.mean(max_count_values)

df = pd.read_csv('filtered_activity.csv')
toy_data = df.head(10)
v_list = []
for i in range(len(df)):
    try:
        v_list.append(find_suit_value(df, i))   
    except:
        v_list.append(99999)
    if i % 10000 == 0:
        print(i)

In [ ]:
# Data V3
df = pd.read_csv('filtered_activity_s2.csv')
df = df[df['Final Activity']!=None]
df = df[df['Final Activity']!=99999]
df = df[['ChEMBL ID', 'PubChem ID', 'IUPHAR ID', 'Target', 'Activity type',
       'Assay type', 'Unit',  'Activity check annotation', 'Ligand names', 'SMILES (Canonical)_C',
       'SMILES (Canonical)_PC', 'SMILES (Canonical)_B', 'SMILES (Canonical)_I',
       'SMILES (Canonical)_PD', 'Structure check (Tanimoto)', 'Source',
       'Final Activity']]
df = df[df['Structure check (Tanimoto)'].isin(['1 structure', 'match', 'no match (1)'])]
df.to_csv('filtered_activity_s3.csv', index=None)

In [ ]:
# Data V4
df = pd.read_csv('/data2/guyaowen/test/filtered_activity_s3.csv')
v = df[['SMILES (Canonical)_C', 'SMILES (Canonical)_PC', 'SMILES (Canonical)_B',
         'SMILES (Canonical)_I', 'SMILES (Canonical)_PD']].values
smiles = []
for i in v:
    s = False
    for j in i:
        if type(j) == str and s==False:
            smiles.append(j)
            s = True
            continue
df['SMILES'] = smiles
del df['SMILES (Canonical)_C'], df['SMILES (Canonical)_PC'], df['SMILES (Canonical)_B'], df['SMILES (Canonical)_I'], df['SMILES (Canonical)_PD']
df.to_csv('/data2/guyaowen/test/filtered_activity_s4.csv', index=None)

In [ ]:
# Data V5
df[df['Final Activity'] > 0].to_csv('/data2/guyaowen/test/filtered_activity_s5.csv', index=None)

In [ ]:
# Data V6
df = pd.read_csv('/data2/guyaowen/test/filtered_activity_s5.csv')
from rdkit import Chem
smiles = df['SMILES'].value_counts().index
new_smiles = []
for s in smiles:
    try:
        new_smiles.append(Chem.MolToSmiles(Chem.MolFromSmiles(s)))
    except:
        try:
            new_smiles.append(Chem.MolToSmiles(Chem.MolFromSmiles(s, sanitize=False)))
        except:
            new_smiles.append(None)

smiles_dict = dict(zip(smiles, new_smiles))
df['SMILES'] = df['SMILES'].map(lambda x: smiles_dict[x])
df = df.dropna(subset=['SMILES'])
df.to_csv('/data2/guyaowen/test/filtered_activity_s6.csv', index=None)

In [4]:
df = pd.read_csv('/home/gyw/master thesis/data/filtered_activity_s6.csv')
t = df['Target'].value_counts()[
    df['Target'].value_counts() >= 300].index
df = df[df['Target'].isin(t)]
df = df.drop_duplicates(subset=['Target', 'SMILES', 'Final Activity'])
df = df.reset_index(drop=True)
df.to_csv('/home/gyw/master thesis/data/filtered_activity_new.csv', index=None)

/home/gyw/anaconda3/envs/cheminfo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_smi = df['SMILES'].unique()
smi = {}
for idx, i in enumerate(df_smi):
    sdf = f'../data/molecule_structure/{idx}.sdf'
    smi[i] = sdf

In [15]:
pd.DataFrame(np.array([list(smi.keys()), 
                       list(smi.values())]).T, 
             columns=['SMILES', 'Conformer_path']
            ).to_csv('data/molecule_structure.csv', index=None)

In [16]:
df['Conformer_path'] = df['SMILES'].map(lambda x: smi[x])

In [18]:
for t in df['Target'].value_counts().index:
    df[df['Target'] == t].to_csv(f'data/target/{t}.csv', index=None)

In [93]:
len(os.listdir(path='data/molecules'))

972225

In [94]:
len(smi)

972960